This code produces all figures as well as summary statistics reported in the main paper and supplement. It has to be run after the analysis code `OIC-OO v7` and the results processing code `OSM Results Processing`, using the same ControlFile and from the same folder; this code will read the processed .tab files from the analysis, and will fail if they are not available. See main `Analysis & Graphing` folder for `ReadMe` with explanation of ControlFile format and fields.

In [ ]:
import os
import json
import regex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
import matplotlib.cm as cm

from shutil import copy
from distutils.dir_util import copy_tree

In [ ]:
# Read specified controlfile and unpack into variables
controlfilename = input("Enter control file name (with extension):")
cf = json.load(open(controlfilename, 'r'))

for k,v in cf.items():
    exec(k + '=v')

for setting in [analysissettings]:
    for k, v in setting.items():
        exec(k + '=v')

# Initialise base working directory
os.chdir(f"{baserunname}_IterCal")
basedir = os.getcwd()
os.chdir('./Results')
os.makedirs('./Figures', exist_ok=True)

In [ ]:
## PLOT ANNUAL POLICY PROJECTION GRAPHS ##

pollist = ['Bup Providers', 'Bup Prov Capacity', 'Diverted Rx Init', 
           'Fent OD Risk', 'Heroin Init', 'Nx Kits', 'Own Rx Init', 
           'Peer Recovery', 'Return to OUD', 'People with Rx', 'Developing OUD']

annpolproj = pd.read_csv(f'{baserunname}_polprojperc.tab', sep = '\t', index_col=[0,1,2])
annpolproj.columns = annpolproj.columns.astype(float)

ap = annpolproj.loc[:, endyear+2.:]

display(ap)

plt.rc('font', size=12)

# formats = pd.DataFrame({"color":['0.35', 'k'],
#                        "linetype":['-', '--'],
#                        "fillcolor":['0.4','k']},
#                      index = ['Total overdose deaths', 'Total with UD'])

# formats = pd.DataFrame({"color":['darkorchid', 'forestgreen'],
#                        "linetype":['-', '--'],
#                        "fillcolor":['darkorchid','forestgreen']},
#                      index = ['Total overdose deaths', 'Total with UD'])

formats = pd.DataFrame({"color":['crimson', 'dodgerblue'],
                       "linetype":['-', '--'],
                       "fillcolor":['crimson','dodgerblue']},
                     index = ['Total overdose deaths', 'Total with UD'])

for scen in pollist:
    fig1, ax = plt.subplots(figsize=[4.5, 4.2])
    s = ap.loc[scen]
    #lines = [ax.plot(s.loc[var, 'EV']) for var in projvars]
    for var in polvars:
        linecolor = formats.loc[var, 'color']
        linetype = formats.loc[var, 'linetype']
        fillcolor = formats.loc[var, 'fillcolor']
        ax.plot(s.loc[var, 'EV']*100, c=linecolor, linestyle=linetype)
        ax.fill_between(s.columns, s.loc[var, str(0.025)]*100, s.loc[var, str(0.975)]*100, alpha=0.2, 
                        color=fillcolor, edgecolor=None)
#         ax.fill_between(s.columns, s.loc[var, str(0.05)]*100, s.loc[var, str(0.95)]*100, alpha=0.2, 
#                         edgecolor=None)
#         ax.fill_between(s.columns, s.loc[var, str(0.1)]*100, s.loc[var, str(0.9)]*100, alpha=0.2, 
#                         edgecolor=None)
#         ax.fill_between(s.columns, s.loc[var, str(0.25)]*100, s.loc[var, str(0.75)]*100, alpha=0.2, 
#                         edgecolor=None)
        ax.plot([2022, 2032], [0, 0], 'k', linestyle=':', lw=1) # reference line at y=0
        ax.set_ylim([-15, 2])
        ax.yaxis.set_major_formatter(mticker.PercentFormatter(decimals=0))
        loc = mticker.MultipleLocator(base=1)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelrotation=90)
    fig1.tight_layout()
    fig1.savefig(f'./Figures/{scen}_95.svg')
    fig1.clf()

In [ ]:
## PLOT CUMULATIVE FENTANYL GRAPHS ##
outcomes = ['Annual opioid overdose deaths', 'Annual prevalence of OUD', 
            'Cumulative opioid overdose deaths', 'Cumulative OUD Person-Years']
fent = ['low fent', 'high fent', 'base fent']

lofent = pd.read_csv(f'{baserunname}_FentL_PolRes.tab', sep='\t', index_col=[1, 0])
hifent = pd.read_csv(f'{baserunname}_FentH_PolRes.tab', sep='\t', index_col=[1, 0])
basefent = pd.read_csv(f'{baserunname}_Base_PolRes.tab', sep='\t', index_col=[1, 0])

display(basefent)

lf = lofent.loc['Baseline'].loc['Projected cumulative UD person years':'Projected total with UD']
hf = hifent.loc['Baseline'].loc['Projected cumulative UD person years':'Projected total with UD']
bf = basefent.loc['Baseline'].loc['Projected cumulative UD person years':'Projected total with UD']

allf = pd.concat([lf, hf, bf], keys = ['low fent', 'high fent', 'base fent']).swaplevel(i=0, j=1, axis=0)

allf.rename(index={'Projected cumulative overdose deaths':'Cumulative opioid overdose deaths',
                   'Projected cumulative UD person years': 'Cumulative OUD Person-Years',
                   'Projected total overdose deaths':'Annual opioid overdose deaths',
                   'Projected total with UD':'Annual prevalence of OUD'
                  }, inplace=True)
allf.index.set_names('Fentanyl Level', level=1, inplace=True)


formats = pd.DataFrame({"color":['tab:blue', 'tab:orange', 'k'],
                       "linetype":['-', '-', '--']},
                     index = ['low fent', 'high fent', 'base fent'])

fig4, axs = plt.subplots(2, 2, figsize=[12, 10], sharex=True, constrained_layout=True) 

for ax, var in zip(axs.flatten(), outcomes):
    s = allf.loc[var]
    for scen in fent:
        linecolor = formats.loc[scen, 'color']
        linetype = formats.loc[scen, 'linetype']
        ax.plot(s.loc[scen], c=linecolor, linestyle=linetype, label=scen)
        loc = mticker.MultipleLocator(base=4)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelrotation=90)
        ax.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}'))
    ax.set_ylabel(str(var))
fig4.legend(['low fent', 'high fent', 'base fent'], loc='center right', bbox_to_anchor = (1.13, 0.52))
fig4.savefig(f'Figures/fentscenarios.svg', bbox_inches='tight')